<a href="https://colab.research.google.com/github/TailUFPB/storIA/blob/main/redditScrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Para treinar o modelo, será necessário baixar uma boa quantidade os posts da sub desejada do Reddit. Precisamos dos posts e da popularidade de cada post. 

A API do próprio Reddit já permite isso. Para utilizá-la é necessário criar uma conta e solicitar acesso a API(https://www.reddit.com/dev/api/).
Depois de criada a conta, utilizamos a biblioteca PRAW(https://praw.readthedocs.io/en/stable/) e entramos com os dados de autenticação da gerados na etapa anterior. 
A API deve retornar os posts que irão compor o dataframe que será utilizado para treinar o modelo. Etretanto existe uma limitação para 1000 posts. Temos duas opções: podemos pegar os 1000 mais populares ou podemos pegar os 1000 mais recentes. Outra opção que a API fornece é retornar o post referente a algum id específico. 

Uma das ideias para conseguir baixar mais do que essa limitação foi usar a função random da API, que retorna um post aleatório. Depois disso, repetiríamos a função random X vezes até conseguir uma quantidade satisfatória de posts. 
O random costumava retornar o mesmo grupo de posts, mesmo que fosse executado milhares de vezes. 

A solução encontrada foi utilizar uma outra API('https://api.pushshift.io/reddit/search/submission/) para criar uma lista com os IDs dos posts, e posteriormente utiliza-la na primeira API para resgatar os textos.

Essa segunda API também contava com limitações nas respostas, então fomos pegando a lista de todos os posts de cada dia nos últimos dez anos. Muitas vezes a API dava erro e não retornava post nenhum para algum dia específico, mas ao tentar esse dia novamente, a API retornava os resultados normalmente. Então cada dia que retornava erro era salvo em uma lista, e seria executado novamente em um outro momento. 

Com a lista de todos os Ids pronta, solicitamos os posts daqueles IDS. Como a sub tem vários anos, alguns dos posts mais antigos tinham sido deletados ou removidos. Foi necessário checar se haviam posts duplicados também.

Por fim, esse mesmo processo foi executado para as subs shortscarystories, scarystories e nosleep.





In [ ]:
!pip install praw -q
import praw
import pandas as pd
from tqdm import tqdm
import requests
import datetime
from joblib import Parallel, delayed

In [ ]:
folder = '/content/drive/MyDrive/dados/nlp2/'
subred = 'nosleep'
#subred = 'poetry'
def main(years = 10):
  !mkdir '$folder$subred/' -p
  print('1st runt IDS: ' + str(getIds(years)))
  n = getErrors()
  print(n)
  print('TOTAL IDS: ' + str(mergeIDS(n)))
  n = getPosts()
  print(n)
  print(mergePosts(n))

In [ ]:
def getIds(years):
  data = int(datetime.datetime.today().timestamp())
  link = 'https://api.pushshift.io/reddit/search/submission/?subreddit=' + subred + '&sort=desc&sort_type=created_utc&after={}&before={}&size=1000&fields=id'
  ids = []
  errors = []
  dias = 86400
  for i in tqdm(range(365*years)):
    try: 
      r = requests.get(link.format(data - dias, data), timeout = 5)
      j = r.json()
      for post in r.json()['data']:
        ids.append(post['id'])
    except:
      errors.append(data)
    data = data - dias
  print(len(ids))
  errors = pd.DataFrame({'errors':errors})
  errors.to_csv('{}{}/errors.csv'.format(folder,subred), index = False)
  df = pd.DataFrame({'id':ids})
  df.to_csv('{}{}/ids.csv'.format(folder,subred), index = False)
  return len(df)


def getErrors():
  errors = []
  for n in range(0,100):
    if (len(errors) == 0):
      try:
        errors = pd.read_csv('{}{}/errors.csv'.format(folder,subred))
      except:
        break
    errors = errors['errors'].tolist()
    link = 'https://api.pushshift.io/reddit/search/submission/?subreddit=' + subred + '&sort=desc&sort_type=created_utc&after={}&before={}&size=1000&fields=id'
    ids = []
    errorsf = []
    dias = 86400
    for i in tqdm(errors):
      try: 
        r = requests.get(link.format(i - dias, i), timeout = 10)
        j = r.json()
        for post in r.json()['data']:
          ids.append(post['id'])
      except:
        errorsf.append(i)
    print(len(ids))
    print(len(errorsf))
    if len(errorsf) == 0:
      break
    errors = pd.DataFrame({'errors':errorsf})
    errors.to_csv('{}{}/errors{}.csv'.format(folder,subred,n+1), index = False)
    df = pd.DataFrame({'id':ids})
    df.to_csv('{}{}/ids{}.csv'.format(folder,subred,n+1), index = False)
  return(n)

def mergeIDS(n):
  df = pd.read_csv('{}{}/ids.csv'.format(folder,subred))
  for i in range(1,n+1):
    df = df.append(pd.read_csv('{}{}/ids{}.csv'.format(folder,subred,i)))
  df.to_csv('{}{}/allIDS.csv'.format(folder,subred), index = False)
  return len(df)

def getPosts():
  import praw
  reddit = praw.Reddit(client_id='s_3FiiAEEguClw', client_secret='eEAenY2x2lyJU6TLlgOvHLMj7JluKQ', user_agent='storIA',check_for_async=False)
  ml_subreddit = reddit.subreddit(subred)
  df = pd.read_csv('{}{}/allIDS.csv'.format(folder,subred))
  lista = df['id'].tolist()
  def capture(part):
    posts = []
    cs = 1000
    for i in range(cs*part, cs*(part+1)):
      try: 
        post = reddit.submission(lista[i])
        posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
      except:
        print('over?')
    posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
    posts.to_csv('{}{}/posts{}.csv'.format(folder,subred,part), index = False)
  Parallel(n_jobs=8)(delayed(capture)(part) for part in tqdm(range(len(df)//1000 +2)))
  return (len(df)//1000 +1)

def mergePosts(n):
  #junta posts
  df = pd.read_csv('{}{}/posts0.csv'.format(folder,subred))
  for i in range(1,n):
    df = df.append(pd.read_csv('{}{}/posts{}.csv'.format(folder,subred,i)))
  df.to_csv('{}{}/allPOSTS{}.csv'.format(folder,subred,subred), index = False)
  return(df.shape)


main(14)

  0%|          | 2/4727 [00:00<05:02, 15.60it/s]

1
1st runt IDS: 1


  0%|          | 2/4363 [00:00<05:00, 14.51it/s]

1
4363


  0%|          | 2/3991 [00:00<05:01, 13.25it/s]

...

100%|██████████| 1/1 [00:00<00:00,  7.76it/s]


0
1
0
0
85


100%|██████████| 2/2 [00:00<00:00, 123.39it/s]

TOTAL IDS: 23


1
(23, 8)
